In [13]:
import pandas as pd
import numpy as np
import os

In [14]:
# --- 1. CONFIGURACIÓN DE RUTAS Y CARGA ---
# Usamos 'os.path.join' para crear rutas que funcionen en cualquier sistema operativo.
INPUT_PATH = os.path.join('01_data', 'input', 'Online Retail.csv')
OUTPUT_PATH = os.path.join('01_data', 'output', 'master_df.csv')

print("Ruta de entrada:", INPUT_PATH)

try:
    # Carga del dataset con el encoding correcto
    # Importante: Asegúrate que el archivo se llame 'Online Retail.csv'
    df = pd.read_csv(INPUT_PATH, encoding='ISO-8859-1')
    print("Dataset cargado exitosamente.")
except Exception as e:
    print(f"ERROR: No se pudo cargar el archivo. Verifica la ruta y el nombre (debe ser Online Retail.csv). Detalle: {e}")
    # Si hay error, detenemos la ejecución aquí.

Ruta de entrada: 01_data\input\Online Retail.csv
Dataset cargado exitosamente.


In [15]:
# --- 2. LIMPIEZA DE DATOS ---
# Eliminar filas con valores nulos (principalmente CustomerID)
df.dropna(subset=['CustomerID'], inplace=True)
# Eliminar transacciones de devolución/cancelación (Quantity < 0)
df = df[df['Quantity'] > 0]
# Asegurar tipos de datos
df['CustomerID'] = df['CustomerID'].astype(int)
df['UnitPrice'] = df['UnitPrice'].astype(float)
df['Quantity'] = df['Quantity'].astype(int)

print(f"Número de registros después de la limpieza: {len(df)}")

Número de registros después de la limpieza: 397924


In [16]:
# --- 3. FEATURE ENGINEERING Y RE-CONTEXTUALIZACIÓN GEOGRÁFICA Y DE MONEDA ---
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Month'] = df['InvoiceDate'].dt.month
df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek # Lunes=0, Domingo=6

# Mapeo estratégico de países y Tasas de Conversión Simuladas
# Tasas simuladas: Multiplicador del precio original (GBP) para simular la moneda local.
conversion_rates = {
    'United Kingdom': {'new_country': 'MEXICO', 'rate': 25.0, 'currency': 'MXN'}, 
    'Germany': {'new_country': 'COLOMBIA', 'rate': 4500.0, 'currency': 'COP'},
    'France': {'new_country': 'CHILE', 'rate': 1000.0, 'currency': 'CLP'},
    'EIRE': {'new_country': 'PERU', 'rate': 4.0, 'currency': 'PEN'},
    'Spain': {'new_country': 'ARGENTINA', 'rate': 900.0, 'currency': 'ARS'},
    'Netherlands': {'new_country': 'BRASIL', 'rate': 6.0, 'currency': 'BRL'},
}

# Crear las nuevas columnas de País y Moneda
df['Original_Country'] = df['Country']
df['Country'] = df['Original_Country'].apply(
    lambda x: conversion_rates.get(x, {}).get('new_country', 'Otro_LATAM')
)
# ************** COLUMNA SOLICITADA **************
df['Local_Currency'] = df['Original_Country'].apply(
    lambda x: conversion_rates.get(x, {}).get('currency', 'USD')
)
# ************************************************

# Aplicar la conversión del precio a la moneda local simulada
def convert_price(row):
    original_country = row['Original_Country']
    rate = conversion_rates.get(original_country, {}).get('rate', 1.0)
    return row['UnitPrice'] * rate

df['UnitPrice_Local'] = df.apply(convert_price, axis=1)

print("Ajuste de moneda simulado completado.")



Ajuste de moneda simulado completado.


In [17]:
# --- 4. AGREGACIÓN DE DEMANDA Y SIMULACIÓN DE COMPETENCIA ---
# A. Crear subconjunto de productos TOP
top_items = df.groupby('StockCode')['Quantity'].sum().nlargest(50).index
df_filtered = df[df['StockCode'].isin(top_items)]

# B. Agregación: Crear la tabla maestra de modelado
# Incluimos 'Local_Currency' como variable de agrupación
df_master = df_filtered.groupby(
    ['StockCode', 'Country', 'Local_Currency', 'InvoiceDate', 'Month', 'DayOfWeek', 'UnitPrice_Local']
).agg(
    Total_Quantity_Sold=('Quantity', 'sum')
).reset_index().rename(columns={'UnitPrice_Local': 'UnitPrice'}) # Renombramos para simplicidad

# C. Simulación del Precio de la Competencia (Competitor_Price)
df_master['Competitor_Price'] = np.random.normal(
    loc=df_master['UnitPrice'],
    scale=df_master['UnitPrice'] * 0.05
)
df_master['Competitor_Price'] = df_master['Competitor_Price'].apply(lambda x: max(x, 0.01))

print("\nPrimeras filas del DataFrame Maestro (con precios y moneda local):")
print(df_master[['Country', 'Local_Currency', 'UnitPrice', 'Competitor_Price', 'Total_Quantity_Sold']].head())


Primeras filas del DataFrame Maestro (con precios y moneda local):
    Country Local_Currency  UnitPrice  Competitor_Price  Total_Quantity_Sold
0    BRASIL            BRL       4.98          4.764774                   12
1     CHILE            CLP     750.00        811.741523                   12
2     CHILE            CLP     830.00        815.662069                   36
3     CHILE            CLP     830.00        836.511028                   12
4  COLOMBIA            COP    3375.00       3366.478516                   24


In [18]:
# --- 5. GUARDAR EL RESULTADO ---
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
df_master.to_csv(OUTPUT_PATH, index=False)
print(f"\n✅ FASE 1 COMPLETADA. Archivo master_df.csv guardado en: {OUTPUT_PATH}")


✅ FASE 1 COMPLETADA. Archivo master_df.csv guardado en: 01_data\output\master_df.csv
